<a href="https://colab.research.google.com/github/z-yoscovits/Unfair_tos/blob/main/model_and_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -U gensim
%pip install -U imbalanced-learn

     |████████████████████████████████| 23.9MB 168kB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 215kB 2.8MB/s 
     |████████████████████████████████| 22.3MB 1.6MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
  Found existing installation: imbalanced-learn 0.4.3
    Uninstalling imbalanced-learn-0.4.3:
      Successfully uninstalled imbalanced-learn-0.4.3


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
data=pd.read_csv('/content/drive/MyDrive/data_challenge/Terms_of_service - terms_of_service.csv')

In [3]:
#Remove sentences that have both labels
counts=data.drop_duplicates().sentence1.value_counts()
contradiction=list(counts[counts>1].index)
data=data[~data.sentence1.isin(contradiction)]

In [4]:
# There are two Sentences with both labels
contradiction

["if you repeatedly infringe other people 's intellectual property rights , we will disable your account when appropriate .",
 '#NAME?']

In [5]:
data.label.value_counts()

0    8363
1    1030
Name: label, dtype: int64

In [6]:
data.drop_duplicates().label.value_counts()

0    8198
1    1014
Name: label, dtype: int64

In [7]:
#There are a handfull of duplicate sentences, removing them to prevent data leakage (i.e the same sentence in both train and test sets)
data=data.drop_duplicates()

In [8]:
#Create val and dev sets
dev_data, val_data=train_test_split(data, test_size=0.1, random_state=42)


In [9]:
#Split dev set into train and test sets, X is kept as a single column dataframe, rather than a Series because that is what imbalanced-learn expects
X_train, X_test, y_train, y_test =train_test_split(dev_data[['sentence1']], dev_data.label, test_size=0.2, random_state=42)

In [10]:
#Over sample minority class to produce balanced training set, leave test set and val set unbalanced so metrics are fair
from imblearn.over_sampling import RandomOverSampler
from sklearn.utils import shuffle
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
X_train,y_train=shuffle(X_resampled, y_resampled)

In [11]:
#Turn 1-column dataframes back into series
X_train=X_train.sentence1
X_test=X_test.sentence1

In [12]:
from keras.preprocessing.text import Tokenizer
import gensim

/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [13]:
#Load custom trained word2vec
w2v = gensim.models.Word2Vec.load('/content/drive/MyDrive/data_challenge/word2vec.model')
vocab = w2v.wv.index_to_key  
tokenizer = Tokenizer()

In [14]:
vocab_size = len(vocab) + 1
tokenizer.fit_on_texts(vocab)

In [15]:
import tensorflow as tf

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, LSTM, Dense, Flatten, AveragePooling1D, Dropout, LSTM, Bidirectional, Embedding, Concatenate, Reshape, Subtract, Dot
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras import regularizers
import re

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [16]:
from gensim import utils

In [17]:
def prepare_inputs(x_sentences, sequence_length):
  x_preprocessed=x_sentences.map(utils.simple_preprocess)
  x_encoded=tokenizer.texts_to_sequences(x_preprocessed)
  x_input = np.asarray(pad_sequences(x_encoded,maxlen=sequence_length, padding='pre'))
  return x_input


In [18]:
max_len=75

input_train = prepare_inputs(X_train, max_len)
input_test = prepare_inputs(X_test, max_len)

In [19]:
#Initialize embedding layer with word2vec model
import numpy as np

def get_weight_matrix():
    # define weight matrix dimensions with all 0
    weight_matrix = np.zeros((vocab_size, w2v.vector_size))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for i in range(len(vocab)):
        weight_matrix[i + 1] = w2v.wv.get_vector(vocab[i], norm=True)
    return weight_matrix

embedding_vectors = get_weight_matrix()

In [65]:
tf.keras.backend.clear_session() 
emb_layer = Embedding(vocab_size, output_dim=w2v.vector_size, weights=[embedding_vectors], input_length=max_len, trainable=True)

sentence_input=Input(shape=(max_len,))
sentence_embedding=emb_layer(sentence_input)
sentence_embedding=Dropout(0.9)(sentence_embedding)
sentence_lstm=Bidirectional(LSTM(16))(sentence_embedding)
x=Dropout(0.7)(sentence_lstm)
x=Dense(16, activation='relu')(x)
x=Dropout(0.7)(x)
output=Dense(1, activation='sigmoid')(x)

model = Model(sentence_input,output)

optimizer = Adam(learning_rate=0.0001)

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [66]:
epochs=10
batch_size=512
model.fit(input_train, y_train, epochs=epochs, batch_size=batch_size,  validation_data=(input_test, y_test))

Epoch 1/10
24/24 [==============================] - 10s 318ms/step - loss: 0.7052 - precision: 0.5056 - recall: 0.5071 - val_loss: 0.6900 - val_precision: 0.1041 - val_recall: 0.3242
Epoch 2/10
24/24 [==============================] - 7s 286ms/step - loss: 0.7054 - precision: 0.4968 - recall: 0.5178 - val_loss: 0.6923 - val_precision: 0.1047 - val_recall: 0.4286
Epoch 3/10
24/24 [==============================] - 7s 281ms/step - loss: 0.7030 - precision: 0.5034 - recall: 0.5376 - val_loss: 0.6939 - val_precision: 0.1111 - val_recall: 0.5440
Epoch 4/10
24/24 [==============================] - 7s 283ms/step - loss: 0.6988 - precision: 0.5098 - recall: 0.5477 - val_loss: 0.6952 - val_precision: 0.1135 - val_recall: 0.6484
Epoch 5/10
24/24 [==============================] - 7s 280ms/step - loss: 0.7042 - precision: 0.4922 - recall: 0.5416 - val_loss: 0.6961 - val_precision: 0.1142 - val_recall: 0.7363
Epoch 6/10
24/24 [==============================] - 7s 284ms/step - loss: 0.7026 - precis

In [67]:
epochs=100
batch_size=512
model.fit(input_train, y_train, epochs=epochs, batch_size=batch_size,  validation_data=(input_test, y_test))

Epoch 1/100
24/24 [==============================] - 7s 287ms/step - loss: 0.6973 - precision: 0.5060 - recall: 0.6032 - val_loss: 0.6993 - val_precision: 0.1123 - val_recall: 0.9615
Epoch 2/100
24/24 [==============================] - 7s 281ms/step - loss: 0.6974 - precision: 0.5103 - recall: 0.6229 - val_loss: 0.6996 - val_precision: 0.1110 - val_recall: 0.9670
Epoch 3/100
24/24 [==============================] - 7s 281ms/step - loss: 0.6957 - precision: 0.5067 - recall: 0.6180 - val_loss: 0.6997 - val_precision: 0.1110 - val_recall: 0.9780
Epoch 4/100
24/24 [==============================] - 7s 281ms/step - loss: 0.6967 - precision: 0.5054 - recall: 0.6195 - val_loss: 0.6997 - val_precision: 0.1118 - val_recall: 0.9890
Epoch 5/100
24/24 [==============================] - 7s 281ms/step - loss: 0.6962 - precision: 0.5011 - recall: 0.6161 - val_loss: 0.6999 - val_precision: 0.1124 - val_recall: 1.0000
Epoch 6/100
24/24 [==============================] - 7s 283ms/step - loss: 0.6967 - p

In [68]:
epochs=100
batch_size=512
model.fit(input_train, y_train, epochs=epochs, batch_size=batch_size,  validation_data=(input_test, y_test))

Epoch 1/100
24/24 [==============================] - 7s 284ms/step - loss: 0.4296 - precision: 0.8517 - recall: 0.8047 - val_loss: 0.3510 - val_precision: 0.4118 - val_recall: 0.8846
Epoch 2/100
24/24 [==============================] - 7s 284ms/step - loss: 0.4306 - precision: 0.8411 - recall: 0.8021 - val_loss: 0.3399 - val_precision: 0.4336 - val_recall: 0.8791
Epoch 3/100
24/24 [==============================] - 7s 284ms/step - loss: 0.4277 - precision: 0.8427 - recall: 0.8097 - val_loss: 0.3487 - val_precision: 0.4171 - val_recall: 0.8846
Epoch 4/100
24/24 [==============================] - 7s 281ms/step - loss: 0.4225 - precision: 0.8504 - recall: 0.8092 - val_loss: 0.3408 - val_precision: 0.4271 - val_recall: 0.8846
Epoch 5/100
24/24 [==============================] - 7s 281ms/step - loss: 0.4189 - precision: 0.8517 - recall: 0.8097 - val_loss: 0.3353 - val_precision: 0.4313 - val_recall: 0.8791
Epoch 6/100
24/24 [==============================] - 7s 282ms/step - loss: 0.4182 - p

In [73]:
filename='/content/drive/MyDrive/data_challenge/lstm_model.tf'
model.save(filename)

INFO:tensorflow:Assets written to: /content/drive/MyDrive/data_challenge/lstm_model.tf/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/data_challenge/lstm_model.tf/assets


In [70]:
import io
import json
tokenizer_json = tokenizer.to_json()
with io.open('/content/drive/MyDrive/data_challenge/tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))

In [20]:
model=tf.keras.models.load_model('/content/drive/MyDrive/data_challenge/lstm_model.tf')

In [24]:
prediction=pd.Series(model.predict(input_test).reshape(-1,))

In [33]:
metrics=pd.DataFrame()


In [34]:
metrics['threshold']=np.arange(0,1,0.01)

In [50]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [32]:
def threshold_prediction(threshold, prediction):
  pred=1 if prediction >= threshold else 0
  return pred

In [43]:
for row in metrics.index:
  metrics.at[row, 'precision']=precision_score(y_test,prediction.map(lambda x: threshold_prediction(metrics.at[row, 'threshold'], x)))

In [47]:
for row in metrics.index:
  metrics.at[row, 'recall']=recall_score(y_test,prediction.map(lambda x: threshold_prediction(metrics.at[row, 'threshold'], x)))

In [51]:
for row in metrics.index:
  metrics.at[row, 'f1']=f1_score(y_test,prediction.map(lambda x: threshold_prediction(metrics.at[row, 'threshold'], x)))

In [52]:
pd.options.display.max_rows = 102
metrics


,threshold,precision,recall,f1
0,0.00,0.109771,1.000000,0.197826
1,0.01,0.216482,0.967033,0.353769
2,0.02,0.234501,0.956044,0.376623
3,0.03,0.246439,0.950549,0.391403
4,0.04,0.258209,0.950549,0.406103
5,0.05,0.265842,0.945055,0.414958
6,0.06,0.276083,0.945055,0.427329
7,0.07,0.285240,0.945055,0.438217
8,0.08,0.291032,0.945055,0.445019
9,0.09,0.301226,0.945055,0.456839


In [53]:
#Check performance with the Validation Data

In [56]:
input_val = prepare_inputs(val_data.sentence1, max_len)

In [57]:
val_prediction=pd.Series(model.predict(input_val).reshape(-1,))

In [59]:
val_metrics=pd.DataFrame()
val_metrics['threshold']=np.arange(0,1,0.01)

In [66]:
for row in val_metrics.index:
  val_metrics.at[row, 'precision']=precision_score(val_data.label,val_prediction.map(lambda x: threshold_prediction(val_metrics.at[row, 'threshold'], x)))
  val_metrics.at[row, 'recall']=recall_score(val_data.label,val_prediction.map(lambda x: threshold_prediction(val_metrics.at[row, 'threshold'], x)))
  val_metrics.at[row, 'f1']=f1_score(val_data.label,val_prediction.map(lambda x: threshold_prediction(val_metrics.at[row, 'threshold'], x)))

In [67]:
val_metrics

,threshold,precision,recall,f1
0,0.00,0.117137,1.000000,0.209709
1,0.01,0.234649,0.990741,0.379433
2,0.02,0.251765,0.990741,0.401501
3,0.03,0.258621,0.972222,0.408560
4,0.04,0.273438,0.972222,0.426829
5,0.05,0.283019,0.972222,0.438413
6,0.06,0.289256,0.972222,0.445860
7,0.07,0.300000,0.972222,0.458515
8,0.08,0.310651,0.972222,0.470852
9,0.09,0.317221,0.972222,0.478360
